--- Model Comparison ---
                        Model F1-Score (Overall) Inference Time (ms/sample) Size (MB)
  afro-xlmr-base (Fine-Tuned)               0.96                      36.53   1074.75
bert-tiny-amharic (Simulated)               0.88                       9.13    107.48


ValueError: zero-size array to reduction operation maximum which has no identity